In [ ]:
# Necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.applications import VGG16

# Check if GPU available
if tf.test.gpu_device_name() == '':
    print('GPU device not found. Using CPU.')
else:
    print('GPU found. Using GPU.')

# Defining deep learning model
def create_transfer_model(input_shape, num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers[:-4]:
        layer.trainable = True

    model = models.Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='sigmoid'))

    return model

# Defining input shape and number of classes
input_shape = (150, 150, 3)
num_classes = 1

# Creating the model
model = create_transfer_model(input_shape, num_classes)

# Compiling the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Setting up data generators with additional data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'test/train',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
    'test/test',
    target_size=input_shape[:2],
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Learning Rate Schedule
def lr_schedule(epoch):
    initial_lr = 0.0001
    if epoch < 5:
        return initial_lr
    else:
        return initial_lr * np.exp(0.1 * (5 - epoch))

lr_scheduler = LearningRateScheduler(lr_schedule)

# Training the model with a higher number of epochs
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

# Saving the model
model.save('/content/save/dental_caries_detection.h5')

# Plotting training and validation accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

# Plotting training and validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

# Evaluate the model on the validation set
validation_generator.reset()
y_true = validation_generator.classes
y_pred_prob = model.predict(validation_generator)
y_pred = np.round(y_pred_prob)  # Round probabilities to get binary predictions

# Confusion Matrix
conf_mat = confusion_matrix(y_true, y_pred)

# Calculate Sensitivity and Specificity
tn, fp, fn, tp = conf_mat.ravel()

sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)

print(f'Sensitivity: {sensitivity:.4f}')
print(f'Specificity: {specificity:.4f}')

# Plotting the Confusion Matrix
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=validation_generator.class_indices,
            yticklabels=validation_generator.class_indices)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')

# Calculate ROC curve for binary classification
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve with AUC value
plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")

# Display AUC value on the plot
plt.annotate(f'AUC = {roc_auc:.2f}', xy=(0.6, 0.2), xytext=(0.6, 0.2), bbox=dict(facecolor='white', alpha=0.5))

plt.show()

# Print Classification Report
class_labels = ['non-caries', 'caries']
print(classification_report(y_true, y_pred, target_names=class_labels))